In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [2]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime


import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}_abi.json") as f:
        data = json.load(f)
        return data

In [3]:

# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine, text
from sqlmodel import Session
from core import constants
from core.db import engine
from datetime import datetime, timedelta

# Create a session
session = Session(engine)

# Define the SQL query to get initiated withdrawals without completions
query = text(
    """
    WITH latest_initiated_withdrawals AS (
        SELECT 
            id,
            from_address,
            to_address,
            tx_hash,
            timestamp,
            input,
            block_number,
            ROW_NUMBER() OVER (PARTITION BY from_address ORDER BY timestamp DESC) AS rn
        FROM public.onchain_transaction_history
        WHERE method_id IN (:withdraw_method_id_1)
        AND to_address = ANY(:vault_addresses)
        AND timestamp >= :start_ts
        AND timestamp <= :end_ts
    ),
    has_later_completion AS (
        SELECT DISTINCT 
            i.from_address,
            i.tx_hash
        FROM latest_initiated_withdrawals i
        WHERE i.rn = 1
        AND EXISTS (
            SELECT 1
            FROM public.onchain_transaction_history c
            WHERE c.from_address = i.from_address
            AND to_address = ANY(:vault_addresses)
            AND c.method_id = :complete_method_id
            AND c.timestamp > i.timestamp
        )
    )
    SELECT *
    FROM latest_initiated_withdrawals i
    WHERE i.rn = 1
    AND NOT EXISTS (
        SELECT 1 
        FROM has_later_completion h 
        WHERE h.from_address = i.from_address
    );
    """
)

# Define parameters for the query
params = {
    "withdraw_method_id_1": "0x12edde5e",  # First initiate withdrawal method ID
    "complete_method_id": "0x4f0cb5f3",     # Complete withdrawal method ID
    "vault_addresses": ["0xF12F8753C25F8005311a24bbbA1f7209249B00e3".lower()],
    "start_ts": int((datetime.now() - timedelta(days=7)).timestamp()),  # Start timestamp is last 7 days
    "end_ts": int(datetime.now().timestamp())  # End timestamp is today
}

# Execute the query and fetch results
with session.begin():
    init_withdraws = session.execute(query, params).all()

# print(init_withdraws)
# Create a DataFrame from the results
# Create a DataFrame from the results
df_withdrawals = pd.DataFrame(init_withdraws, columns=["id", "from_address", "to_address", "tx_hash", "timestamp", "input", "block_number", "rn"])

# Display the DataFrame
df_withdrawals.head()

,id,from_address,to_address,tx_hash,timestamp,input,block_number,rn
0,69260d73-dc47-4f96-bf26-646a2421b7a4,0x055ba87dbff972e23bcf26ea4728c31e05240e66,0xf12f8753c25f8005311a24bbba1f7209249b00e3,0xd27959c62a7420128bdf2335b5a9a5f4a905017ce81c...,1735281340,0x12edde5e000000000000000000000000000000000000...,289060769,1
1,b312af5e-1a38-4ce5-8aa0-bfe9405058d1,0x2d0ba6d0ba814f649ad8db7debfbf2e8d385c541,0xf12f8753c25f8005311a24bbba1f7209249b00e3,0x1c2037d94993c2e7f18c16f4754a473f4ec527af728a...,1735600561,0x12edde5e000000000000000000000000000000000000...,290332492,1
2,adb8e4e0-d564-4e5b-a0e7-1d2b8cc32aeb,0x345575058e0447d5f06a4edd72a5daa39588ce04,0xf12f8753c25f8005311a24bbba1f7209249b00e3,0xf96f0bb608e2a77e5adb10e06153361f5cf65cf8a84c...,1735384171,0x12edde5e000000000000000000000000000000000000...,289470359,1
3,dc57f319-ccb6-4539-9edc-5a3fef4ed375,0xa987ede1c48ef80f7493d467f8cd8e1cf93cf190,0xf12f8753c25f8005311a24bbba1f7209249b00e3,0x6b29b89a965aed53b825f4325c2f31b8c4a5ee98d4a2...,1735603925,0x12edde5e000000000000000000000000000000000000...,290345895,1
4,931d3515-646f-499d-a9e6-a45c7d4cd71c,0xc117204767439576495560a40f66623ac1402f9f,0xf12f8753c25f8005311a24bbba1f7209249b00e3,0xffaf2cb01d8a0b8f6bc24c2d6cbd128a2d0165da6f2a...,1735297324,0x12edde5e000000000000000000000000000000000000...,289124470,1


In [4]:
w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

rockonyx_delta_neutral_vault_abi = read_abi("rockonyxdeltaneutralvault")
vault_contract = w3.eth.contract(
    address="0xF12F8753C25F8005311a24bbbA1f7209249B00e3",
    abi=rockonyx_delta_neutral_vault_abi,
)


In [5]:
def _extract_input(data):
    shares = int(data[10:], 16) / 1e6
    return shares

def get_pps(vault_contract: Contract, decimals = 1e6, block_number=200199390):
    tvl = vault_contract.functions.pricePerShare().call(block_identifier=block_number)

    return tvl / decimals

df_withdrawals['shares'] = df_withdrawals.apply(
    lambda row: _extract_input(
        row['input']
    ),
    axis=1
)

df_withdrawals['pps'] = df_withdrawals.apply(
    lambda row: get_pps(
        vault_contract,
        block_number=int(row['block_number'])
    ),
    axis=1
)

df_withdrawals['amount_usd'] = df_withdrawals['shares'] * df_withdrawals['pps']

df_withdrawals[['from_address', 'shares', 'pps', 'amount_usd', 'block_number']]

,from_address,shares,pps,amount_usd,block_number
0,0x055ba87dbff972e23bcf26ea4728c31e05240e66,98.603318,1.003363,98.934921,289060769
1,0x2d0ba6d0ba814f649ad8db7debfbf2e8d385c541,1097.355309,1.005712,1103.623403,290332492
2,0x345575058e0447d5f06a4edd72a5daa39588ce04,54.594221,1.004796,54.856055,289470359
3,0xa987ede1c48ef80f7493d467f8cd8e1cf93cf190,99.520609,1.005712,100.089071,290345895
4,0xc117204767439576495560a40f66623ac1402f9f,9958.899621,1.003480,9993.556592,289124470


In [8]:
# Function to get withdraw pool amounts
def get_withdraw_pool_amount(vault_contract: Contract):
    try:
        # Call the getWithdrawPoolAmount function
        result = vault_contract.functions.getWithdrawPoolAmount().call()
        
        # Extract scWithdrawPoolAmount and ptWithdrawPoolAmount
        scWithdrawPoolAmount = result / 1e6  # Convert to float
        
        return scWithdrawPoolAmount
    except Exception as e:
        print(f"Error fetching withdraw pool amounts: {e}")
        return None, None
    
withdrawal_pool_amount = get_withdraw_pool_amount(vault_contract)
print('Withdrawl pool', withdrawal_pool_amount)
print('Need withdraw', df_withdrawals['amount_usd'].sum())

amount_to_withdraw = withdrawal_pool_amount - df_withdrawals['amount_usd'].sum()
if amount_to_withdraw < 0:
    print('amount_to_withdraw', abs(amount_to_withdraw*(1+ (0.01/100))))


Withdrawl pool 11251.164189
Need withdraw 11351.060040767044
amount_to_withdraw 99.90584135222201


In [9]:
data = {"action":"retry_handle_withdrawal","amount_in_usd":abs(round(amount_to_withdraw*(1+ (0.01/100)),2)),"from_addresses":None}
print(json.dumps(data))

{"action": "retry_handle_withdrawal", "amount_in_usd": 99.91, "from_addresses": null}
